Метрика соревнования — ROC-AUC – для определения пола, f1 weighted – для определения возраста. Все решения высчитываются по формуле - 2 * f1_weighted(по 6 возрастным бакетам) + gini по полу. Возрастные бакеты 18-24, 25-34, 35-44, 45-54, 55-64 65+.

In [1]:
import pandas as pd
%pip install polars -q
import polars as pl
import numpy as np
import time
%pip install pyarrow -q
import pyarrow.parquet as pq
import scipy
%pip install implicit -q
import implicit
import bisect
import sklearn.metrics as m
%pip install catboost -q
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve, CalibratedClassifierCV

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


/Users/a125x/opt/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Note: you may need to restart the kernel to use updated packages.


In [2]:
LOCAL_DATA_PATH = './context_data/'
SPLIT_SEED = 42
DATA_FILE = 'competition_data_final_pqt'
TARGET_FILE = 'public_train.pqt'
SUBMISSION_FILE = 'submit_2.pqt'

In [3]:
id_to_submit = pq.read_table(f'{LOCAL_DATA_PATH}/{SUBMISSION_FILE}').to_pandas()
id_to_submit.head()

,user_id
221301,221301
31271,31271
211594,211594
253119,253119
192578,192578


In [4]:
data_example = pq.read_table(f'{LOCAL_DATA_PATH}/{DATA_FILE}/part-00000-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet')
pd.DataFrame([(z.name, z.type) for z in data_example.schema], columns = [['field', 'type']])

,field,type
0,region_name,string
1,city_name,string
2,cpe_manufacturer_name,string
3,cpe_model_name,string
4,url_host,string
5,cpe_type_cd,string
6,cpe_model_os_type,string
7,price,double
8,date,date32[day]
9,part_of_day,string


In [5]:
df = pl.DataFrame(list(data_example)) 

In [9]:
ls = list(data_example)
ls

 [
   [
     "Краснодарский край",
     "Краснодарский край",
     "Краснодарский край",
     "Краснодарский край",
     "Краснодарский край",
     ...
     "Республика Северная Осетия — Алания",
     "Республика Северная Осетия — Алания",
     "Республика Северная Осетия — Алания",
     "Республика Северная Осетия — Алания",
     "Республика Северная Осетия — Алания"
   ],
   [
     "Республика Северная Осетия — Алания",
     "Республика Северная Осетия — Алания",
     "Республика Северная Осетия — Алания",
     "Республика Северная Осетия — Алания",
     "Республика Северная Осетия — Алания",
     ...
     "Кировская область",
     "Кировская область",
     "Кировская область",
     "Кировская область",
     "Кировская область"
   ],
 ...,
   [
     "Новосибирская область",
     "Новосибирская область",
     "Новосибирская область",
     "Новосибирская область",
     "Новосибирская область",
     ...
     "Краснодарский край",
     "Краснодарский край",
     "Краснодарский край",
   